In [49]:
import fastf1
import pandas as pd
import numpy as np

session = fastf1.get_session(2019, 15, 'Race')
session.load()
laps = session.laps


core           INFO 	Loading data for Singapore Grand Prix - Race [v3.1.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No 

In [50]:
# get information on the observed car's previous follower
# this follower will most likely be the same as the 'pursuer', but that is not guaranteed
# 
# example. The car we're following is LEC, if the order during lap i-1 is HAM(3rd) NOR(2nd), LEC(1st). 
# Then on lap i-1, Norris pits. Now on lap i LEC only has information on HAM's movement.
# I want to include info about Norris. thus LEC's risk of an undercut
# input: index of the observed car
# outputs: 
#  - prev_follower_pitted (boolean) 1 if the previous follower pitted, 0 else
#  - prev_follower_stint: (int) prev followers stint
#  - prev_follower_pace: (float) prev followers sector two time in seconds
def prev_follower_pitted_funct(ind):
    # gets the observed car's previous lap and their position.
    driver_num = laps.loc[ind, "DriverNumber"]
    lap_num = laps.loc[ind, "LapNumber"]
    
    # get info on previous follower
    if(lap_num == 1 or lap_num == 2):
        # the driver was in 20th place, and thus does not have a follower. Sad. OR
        # this is the second lap, so the prev lap (lap 1) has no real information
        # this is the first lap, and there is no previous lap
        return np.nan, np.nan, np.nan

    orig_driver_prev_pos = laps.pick_lap(lap_number = lap_num - 1).loc[laps['DriverNumber'] == driver_num]["Position"].iloc[0]
    if(orig_driver_prev_pos == 20): 
        return np.nan, np.nan, np.nan
    
    prev_follower = laps.pick_lap(lap_number=lap_num - 1).loc[laps['Position'] == (orig_driver_prev_pos + 1)]

    # gets prevous follower's information    
    prev_follower_pitted = 0
    try:
        if ((not pd.isnull(prev_follower["PitInTime"].iloc[0])) and lap_num != 1.0):
            #driver in front goes in pit (excluding first lap)
            prev_follower_pitted = 1
    except:
        # this is the same positional problem as earlier
        return np.nan, np.nan,np.nan
    prev_follower_stint = int (prev_follower["Stint"].iloc[0])
    prev_follower_pace = prev_follower["LapTime"].iloc[0].total_seconds()
    

    return prev_follower_pitted, prev_follower_stint, prev_follower_pace
    
# returns dictonary of the drivers fastest laps
#     key is the driver number
#     value is time in seconds 
def drivers_fastest_laps(laps):
    drivers_fastest = {}
    for un_d in laps["DriverNumber"].unique():
        # print(un_d)
        fastest_lap_time=laps.pick_drivers(un_d).pick_fastest()["LapTime"].total_seconds()
        drivers_fastest[un_d] = fastest_lap_time
    # laps[0:40].pick_fastest()
    return drivers_fastest

# input: index of lap to get info from
# output: in order:
#    -  in_front_pace:(float) gets the time pursuer took to get to sect2_time in seconds
#    - pursuer_time: (pandas...Timedelta) gets the pursuers_sect2_time
#    - in_front_stint: (int) the stint of the car in front
#    - in_front_pited:(boolean) did the car in front pit?
#    - 
def get_in_front_info(ind):
    pos =  laps.loc[ind, "Position"]
    lap_num = laps.loc[ind, "LapNumber"]
    if pos == 1 or pos == None:
        # this removes the 1st driver from consideration bc they dont have a follower to act on
        return np.nan, np.nan, np.nan, np.nan
    driver_in_front = laps.pick_lap(lap_number=lap_num).loc[laps['Position'] == (pos - 1)]
    if driver_in_front.empty:
        return np.nan, np.nan, np.nan, np.nan
    
    in_front_pitted = 0
    if ((not pd.isnull(driver_in_front["PitInTime"].iloc[0])) and lap_num != 1.0):
        #driver in front goes in pit (excluding first lap)
        in_front_pitted = 1
    in_front_stint = int (driver_in_front["Stint"].iloc[0])
    in_front_pace = driver_in_front["LapTime"].iloc[0].total_seconds()
    in_front_time = driver_in_front["Sector2SessionTime"].iloc[0]
    return in_front_pace, in_front_time, in_front_stint, in_front_pitted

# input: index of driver to get info from
# output: in order:
#    -  pursuer_pace: (float) gets the time pursuer took to get to sect2_time in seconds
#    - pursuer_time:(pandas...Timedelta) gets the pursuers_sect2_time
#    - pursuer_stint: (int) pursuers stint
def get_pursuer_info(ind):
    pos =  laps.loc[ind, "Position"]
    lap_num = laps.loc[ind, "LapNumber"]
    if pos == 20:
        # this removes the 20th driver from consideration (which shouldnt be included in final product)
        return np.nan, np.nan, np.nan
    
    try:
        driver_following = laps.pick_lap(lap_number=lap_num).loc[laps['Position'] == (pos + 1)]
        pursuer_stint = int (driver_following["Stint"].iloc[0])

    except:
        # throws an error when soemone is not in 19th place, but the 20th place person has been knocked out
        # so 19 is last place 
        return np.nan, np.nan, np.nan
    # pursuer_stint = int (driver_following["Stint"].iloc[0])
    pursuer_pace = driver_following["LapTime"].iloc[0].total_seconds()
    pursuer_time = driver_following["Sector2SessionTime"].iloc[0]
    return pursuer_pace, pursuer_time, pursuer_stint

def get_driver_num(ind):
    return laps.loc[ind, "DriverNumber"]



In [51]:
col_names = ['prev_follower_pitted', 'prev_follower_stint', 'prev_follower_pace',
             'in_front_pace', 'in_front_time', 'in_front_stint', 'in_front_pitted',
             'pursuer_pace', 'pursuer_time', 'pursuer_stint',
             ]


laps_pd = pd.DataFrame(laps)

for name in col_names:
    laps_pd[col_names] = np.nan

fastest_laps = drivers_fastest_laps(laps)
fastest_lap_of_race = laps.pick_fastest()["LapTime"].total_seconds()

laps_pd['fastest_laps'] = np.nan
laps_pd['fastest_lap_of_race'] = fastest_lap_of_race

for ind in laps.index:
    prev_follower_pitted, prev_follower_stint, prev_follower_pace = prev_follower_pitted_funct(ind)
    in_front_pace, in_front_time, in_front_stint, in_front_pitted = get_in_front_info(ind)
    pursuer_pace, pursuer_time, pursuer_stint = get_pursuer_info(ind)
    
    var_names = [ prev_follower_pitted, prev_follower_stint, prev_follower_pace,
             in_front_pace, in_front_time, in_front_stint, in_front_pitted,
             pursuer_pace, pursuer_time, pursuer_stint
             ]
    
    for i in range(len(var_names)):
        laps_pd[col_names[i]].iloc[ind]= var_names[i]
        
    laps_pd['fastest_laps'].iloc[ind] = fastest_laps[get_driver_num(ind)]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


/var/folders/5z/md7jx1px78s11b1rm8_cd_ww0000gn/T/ipykernel_8758/2190254296.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_pd[col_names[i]].iloc[ind]= var_names[i]
/var/folders/5z/md7jx1px78s11b1rm8_cd_ww0000gn/T/ipykernel_8758/2190254296.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_pd['fastest_laps'].iloc[ind] = fastest_laps[get_driver_num(ind)]


16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286


In [52]:
laps_pd

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,prev_follower_pace,in_front_pace,in_front_time,in_front_stint,in_front_pitted,pursuer_pace,pursuer_time,pursuer_stint,fastest_laps,fastest_lap_of_race
0,0 days 00:35:40.246000,GAS,10,0 days 00:01:59.586000,1.0,1.0,0 days 00:00:06.115000,NaT,NaT,0 days 00:00:45.209000,...,NaN,118.793,0 days 00:35:00.753000,1.0,False,120.020,0 days 00:35:01.533000,1.0,105.769,102.301
1,0 days 00:37:31.079000,GAS,10,0 days 00:01:50.833000,2.0,1.0,NaT,NaT,0 days 00:00:29.936000,0 days 00:00:42.990000,...,NaN,110.762,0 days 00:36:52.466000,1.0,False,110.939,0 days 00:36:53.671000,1.0,105.769,102.301
2,0 days 00:39:21.722000,GAS,10,0 days 00:01:50.643000,3.0,1.0,NaT,NaT,0 days 00:00:29.573000,0 days 00:00:43.077000,...,110.939,110.376,0 days 00:38:43.054000,1.0,False,110.595,0 days 00:38:44.136000,1.0,105.769,102.301
3,0 days 00:41:11.754000,GAS,10,0 days 00:01:50.032000,4.0,1.0,NaT,NaT,0 days 00:00:29.580000,0 days 00:00:42.656000,...,110.595,110.103,0 days 00:40:33.219000,1.0,False,110.258,0 days 00:40:34.771000,1.0,105.769,102.301
4,0 days 00:43:01.511000,GAS,10,0 days 00:01:49.757000,5.0,1.0,NaT,NaT,0 days 00:00:29.388000,0 days 00:00:42.594000,...,110.258,109.542,0 days 00:42:22.874000,1.0,False,110.073,0 days 00:42:24.837000,1.0,105.769,102.301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,0 days 02:25:28.268000,GIO,99,0 days 00:01:46.004000,57.0,3.0,NaT,NaT,0 days 00:00:28.722000,0 days 00:00:41.159000,...,107.082,105.883,0 days 02:24:50.608000,3.0,False,107.678,0 days 02:24:56.165000,3.0,105.630,102.301
1161,0 days 02:27:14.331000,GIO,99,0 days 00:01:46.063000,58.0,3.0,NaT,NaT,0 days 00:00:28.804000,0 days 00:00:41.136000,...,107.678,106.058,0 days 02:26:36.752000,3.0,False,107.563,0 days 02:26:44.030000,3.0,105.630,102.301
1162,0 days 02:28:59.961000,GIO,99,0 days 00:01:45.630000,59.0,3.0,NaT,NaT,0 days 00:00:28.603000,0 days 00:00:40.899000,...,107.563,105.765,0 days 02:28:22.476000,3.0,False,107.572,0 days 02:28:31.507000,3.0,105.630,102.301
1163,0 days 02:30:45.994000,GIO,99,0 days 00:01:46.033000,60.0,3.0,NaT,NaT,0 days 00:00:28.925000,0 days 00:00:41.058000,...,107.572,105.934,0 days 02:30:08.290000,3.0,False,108.451,0 days 02:30:19.647000,3.0,105.630,102.301


In [53]:
print(len(laps_pd))

1165
